In [112]:
from connection import ConnectionTask, pg_select
import time
import tablib
import requests
import json
import random
import arrow
import re
from pprint import pprint
c=ConnectionTask()

In [ ]:
# 51返呗Id：664的滑动，已领卷但是未使用的用户注册手机号。
app_code='fdbank'
activity_id='1248'
expire_time = time.mktime((2018,3,26,0,0,0,0,0,0))
rdb=c.prdb_activity(0)
user_id_set=set()
count = 0
for key in rdb.scan_iter('c:{}:*'.format(app_code)):
    if 'use' in key:
        continue
    count +=1

    value,timestamp=rdb.zscan(key)[1][0]
    if value.split('|')[-1]==activity_id:
        if timestamp>expire_time:
            continue

        if 'expired' in key:
            user_id_set.add(key.split(':')[-2])
        else:
            user_id_set.add(key.split(':')[-1])
            
    print(count,end='\r')
            
import tablib
user=tablib.Dataset(headers=['用户ID','用户手机号'])
for user_id in user_id_set:
    try:
        phone=c.pmdb.user_customer.find_one({'customerUser.{}'.format(app_code):user_id},{'_id':0,'phone':1})['phone']
    except:
        phone=''
    user.append([user_id,phone])
    print(user_id,end='\r')
with open(f'已领卷未使用的用户手机号{app_code}_{activity_id}.xlsx', 'wb') as f:
    f.write(user.export('xlsx'))
print('ok')

In [ ]:
len(user_id_set)

In [ ]:
import tablib
user=tablib.Dataset(headers=['用户ID','用户手机号'])
for user_id in user_id_set:
    try:
        phone=c.pmdb.user_customer.find_one({'customerUser.{}'.format(app_code):user_id},{'_id':0,'phone':1})['phone']
    except:
        phone=''
    user.append([user_id,phone])
    print(user_id,end='\r')
with open(f'已领卷未使用的用户手机号{app_code}_{activity_id}.xlsx', 'wb') as f:
    f.write(user.export('xlsx'))
print('ok')

In [ ]:
with open('51ala_664.xls', 'wb') as f:
    f.write(user.export('xls'))

In [ ]:
len(user.dict)

In [4]:
# 运营生成卷码\
# 活动id：779  优惠券码760份，id 778，优惠券码3010份
import requests
import tablib
COUPON_URL='http://sessionapi.localdomain:8011/get_coupon_codes/'
# 1251，bsb ，11700个券码
app_code='bsb'
activity_num=[
    {'activity_id':1307,'num':4200},
]

data_book=tablib.Databook()
for activity in activity_num:
    url=COUPON_URL+'{}/{}/{}'.format(app_code,activity['activity_id'],activity['num'])
    resp_data=requests.get(url).json()
#     {
#         'data': {
#             'codes': ['f3e979c46f88c0b5']
#         }, 
#         'message': 'success', 
#         'code': 0
#     }
    print(resp_data['message'])
    dataset=tablib.Dataset(header=['appCode','activityId','couponCode'])
    for item in resp_data['data']['codes']:
        dataset.append([app_code,activity['activity_id'],item])
    dataset.title=str(activity['activity_id'])
    data_book.add_sheet(dataset)
    print(activity,'ok')
with open('coupon-{}-{}.xls'.format(app_code,'-'.join([str(activity['activity_id']) for activity in activity_num])), 'wb') as f:
    f.write(data_book.xls)

success
{'activity_id': 1307, 'num': 4200} ok


{'ets': 1525449599000, 'app_id': 175}


In [26]:
# 运营生成卷码
import requests
import tablib
COUPON_URL='http://promotionapi.localdomain:8011/get_coupon_code_list'
conn = c.ppgdb('oto_saas')

activity_num=[
    {'activityId':1321, 'appCode': 'bsb', 'num':600},
    {'activityId':1323, 'appCode': 'bsb', 'num':400},
]
for an in activity_num:
    for item in pg_select("SELECT ets,app_id FROM activity WHERE id = '{}';".format(an['activityId']),conn):
        item['appId'] = item['app_id']
        an.update(item)

data_book=tablib.Databook()
for activity in activity_num:
    resp_data=requests.post(COUPON_URL,json=activity).json()
#     {
#         'data': {
#             'codes': ['f3e979c46f88c0b5']
#         }, 
#         'message': 'success', 
#         'code': 0
#     }
    print(resp_data['message'])
    dataset=tablib.Dataset(headers=['appCode','activityId','couponCode'])
    for item in resp_data['data']['codes']:
        dataset.append([app_code,activity['activityId'],item])
    dataset.title=str(activity['activityId'])
    data_book.add_sheet(dataset)
    print(activity,'ok')
with open('coupon-{}-{}.xls'.format(app_code,'-'.join([str(activity['activityId']) for activity in activity_num])), 'wb') as f:
    f.write(data_book.xls)

success
{'activityId': 1321, 'appCode': 'bsb', 'num': 600, 'ets': 1527782399000, 'app_id': 175, 'appId': 175} ok
success
{'activityId': 1323, 'appCode': 'bsb', 'num': 400, 'ets': 1527782399000, 'app_id': 175, 'appId': 175} ok


In [ ]:
# 基于mongo.timeline统计ele外卖失败订单原因
import tablib,time

start=time.mktime((2017,11,1,0,0,0,0,0,0))*1000
end=time.mktime((2017,12,12,0,0,0,0,0,0))*1000
counter={}
for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end},'status':{'$nin':[1,2,3,4]}},{'id':1,'timeline':1,'_id':0}):
    for time_line in item.get('timeline',[]):
        if '订单失败' in time_line and '支付超时' not in time_line:
            date,*args,reason=time_line.strip(' ').split(' ',3)
            if len(reason)<=2:
                print(time_line,reason)
            counter.setdefault(date,{})
            counter[date].setdefault(reason,0)
            counter[date][reason]+=1
            break

tab_data=tablib.Dataset(headers=['时间','原因','单数'])
for key,value in counter.items():
    for k,v in value.items():
        tab_data.append([key,k,v])

with open('waimai_refund_order_counter_{}-{}.xlsx'.format(
    time.strftime('%Y%m%d',time.localtime(start/1000)),
    time.strftime('%Y%m%d',time.localtime(end/1000))),'wb') as f:
    f.write(tab_data.xlsx)

In [ ]:
# 基于mongo.timeline统计ele外卖失败订单原因
# 把11.1-12.10号的订单中原因是
# 【商家超时未接单】、【暂时无法配送】
# 这两个原因的订单明细导出一份

import tablib,time

start=time.mktime((2017,11,1,0,0,0,0,0,0))*1000
end=time.mktime((2017,12,10,24,0,0,0,0,0))*1000
counter=[]
for item in c.pmdb.order_waimai.find({'createdAt':{'$gte':start,'$lt':end},'status':{'$nin':[1,2,3,4]}},{'_id':0}):
    for time_line in item.get('timeline',[]):
        if '订单失败' in time_line and '支付超时' not in time_line:
            date,*args,reason=time_line.strip(' ').split(' ',3)
#             if '商家超时未接单' in reason or '暂时无法配送' in reason:
            if '暂时无法配送' in reason:
                counter.append(item['id'])
#             if len(reason)<=2:
#                 print(time_line,reason)
#             counter.setdefault(date,{})
#             counter[date].setdefault(reason,0)
#             counter[date][reason]+=1
            break
# tab_data=tablib.Dataset()
# tab_data.dict=counter
# tab_data=tablib.Dataset(headers=['时间','原因','单数'])
# for key,value in counter.items():
#     for k,v in value.items():
#         tab_data.append([key,k,v])

# with open('waimai_refund_order_counter_{}-{}.xlsx'.format(
#     time.strftime('%Y%m%d',time.localtime(start/1000)),
#     time.strftime('%Y%m%d',time.localtime(end/1000))),'wb') as f:
#     f.write(tab_data.xlsx)

In [ ]:
len(counter) #1415 = 1047 + 368

In [ ]:
# 客户，活动Id ，使用金额，结束时间
tab_data=tablib.Dataset(headers=['客户','活动ID','使用数量','优惠单价','优惠总额','结束时间'])
coupon_counter={}
start=time.mktime((2017,1,1,0,0,0,0,0,0))*1000
for order_type in c.pmdb.order_lite_list.distinct('orderType',{'status':4}):
    for item in c.pmdb[f'order_{order_type}'].find(
        {'status':4,'createdAt':{'$gte':start},'couponId':{'$exists':True}},
        {'id':1,'coupon':1,'couponId':1,'appCode':1,'user_id':1,'_id':0,'customerUserId':1}
    ):
        coupon_counter.setdefault(item['appCode'],{})
        used_coupon_list_key = 'c:{}:{}:used'.format(item['appCode'], item.get('user_id') or item.get('customerUserId'))
        for activity, coupon_ts in c.prdb_activity(0).zscan_iter(used_coupon_list_key, match='{}|*'.format(item['couponId'])):
            
            *_, aid, paid = activity.split('|')
            content = json.loads(c.prdb_activity(0).hget(f'a:{paid}','content'))
            if time.time()<content['ets']:
                continue
            ets = time.strftime("%Y-%m-%d",time.localtime(content['ets']))
            coupon_counter[item['appCode']].setdefault(paid, {"count":0,"unitPrice":0,"endDate":""})
            coupon_counter[item['appCode']][paid]['count']+=1
            coupon_counter[item['appCode']][paid]['unitPrice']=item['coupon'].get('price')
            coupon_counter[item['appCode']][paid]['endDate']=ets

for app_code in coupon_counter:
    for activity_id,value in coupon_counter[app_code].items():
        tab_data.append([app_code,activity_id,value['count'],value['unitPrice'],value['count']*value['unitPrice'],value['endDate']])   

with open('红包活动使用总额.xlsx','wb') as f:
    f.write(tab_data.xlsx)

In [123]:
# 几个账号，查下账户中有没有券
import json,time
# phone_list = '''13774299451
# 13816874115'''.split('\n')

# phone_list = ['18669066600']
# # userid_list = ['201607060002599003','201502270000157548']
# app_code='icbc'

# user_list = {item['phone']:item['customerUser'][app_code] for item in c.pmdb.user_customer.find({'phone':{'$in':phone_list}})}
# print('手机号','用户ID','优惠券')
# for phone,user_id in user_list.items():
    
# #     user_coupon = [
# #         c.prdb_activity(0).hget(
# #             'a:'+item.split('|')[1],'title'
# #         ) for item,_ in c.prdb_activity(0).zscan_iter(f'c:{app_code}:{user_id}')
# #     ]
    
# #     print(phone,user_id,user_coupon)
#     for item,_ in c.prdb_activity(0).zscan_iter(f'c:{app_code}:{user_id}'):
#         title=c.prdb_activity(0).hget('a:'+item.split('|')[1],'title')
#         parentTitle=c.prdb_activity(0).hget('a:'+item.split('|')[1],'parentTitle')
#         parentId=c.prdb_activity(0).hget('a:'+item.split('|')[1],'parentId')
#         content=json.loads(c.prdb_activity(0).hget('a:'+item.split('|')[1],'content'))
#         start_time = time.strftime("%Y-%m-%d %H:%M:%S'",time.localtime(content['sts']))
#         end_time = time.strftime("%Y-%m-%d %H:%M:%S'",time.localtime(content['ets'])) 
#         time_info = str(start_time) + ' ~ ' + str(end_time)
#         print(title,parentTitle,parentId,time_info)
#         break
#     break
phone = '18669066600'
data =c.prdb_activity(0).zscan_iter('c:{}:{}'.format('icbc',phone))
if data:
    print('have')
else:
    print('no')
user_list ={item['phone']:item['customerUser'][app_code] for item in c.pmdb.user_customer.find({'phone':phone})}
if user_list:
    print('have')
else:
    print('no')
print(user_list)
for phone,user_id in user_list.items():
    for item in c.prdb_activity(0).zscan_iter(f'c:{app_code}:{user_id}'):
        print('xxxx : {}'.format(item))
        get_time = time.strftime("%Y-%m-%d %H:%M:%S'",time.localtime(item[1]))
        title=c.prdb_activity(0).hget('a:'+item[0].split('|')[1],'title')
        parentTitle=c.prdb_activity(0).hget('a:'+item[0].split('|')[1],'parentTitle')
        parentId=c.prdb_activity(0).hget('a:'+item[0].split('|')[1],'parentId')
        content=json.loads(c.prdb_activity(0).hget('a:'+item[0].split('|')[1],'content'))
        start_time = time.strftime("%Y-%m-%d %H:%M:%S'",time.localtime(content['sts']))
        end_time = time.strftime("%Y-%m-%d %H:%M:%S'",time.localtime(content['ets'])) 
        time_info = str(start_time) + ' ~ ' + str(end_time)
        print(title,parentTitle,parentId,time_info,'get_time:{}'.format(get_time))

have
have
{'18669066600': '502D5587B0697DAD406EC09760C904A7FE1C3C226FC087E610553CE10ADBB23744F4BA3DB4B1F50A'}
xxxx : ('27828473|c:1076|1589', 1531380843.0)
白名单测试红包 云南分行测试活动 1589 2018-07-13 00:00:00' ~ 2018-07-15 23:59:59' get_time:2018-07-12 15:34:03'


In [16]:
a=1
b=2
c={"a":a,"b":b}
'a:{},b:{}'.format(a,b)

'a:1,b:2'

In [17]:
f'a:{c["a"]},b:{b}'

'a:1,b:2'

In [ ]:
# 分城市的有排期的影院列表

In [ ]:
# 3个需求
# 1.统计一下翼支付猫眼电影上线至3.13号的出票张数（一个订单可能会有多张票）
# 2.统计最近1万单话费和流量充值的支付成功到充值成功的时间间隔，1分钟以内 X%，1-5分钟 X%，5-30分钟 X%，30分钟以上 X%
# 3.统计最近1万单话费和流量充值的失败原因占比，A原因X%，B原因X%，未知原因X%

In [ ]:
count=0
for item in c.pmdb.order_dianying.find({'channel':'maoyan','status':4,'appCode':'bestpay'},{'seatName':1,'_id':0}):
    count+= len(item['seatName'].split(' '))
print(count)

In [ ]:
time_cost={}
count=0
for item in c.pmdb.order_huafei.find({'status':4},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    try:
        for timeline in item['timeline_new']:
            if timeline['status']=='充值中':
                time_1 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
            elif timeline['status']=='充值成功':
                time_2 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
    except :
        continue
            
    cost = time_2-time_1
    if cost <= 60:
        time_cost.setdefault('~1',0)
        time_cost['~1']+=1
    elif 60< cost <= 60*5:
        time_cost.setdefault('1~5',0)
        time_cost['1~5']+=1
    elif 60*5< cost <= 60*30:
        time_cost.setdefault('5~30',0)
        time_cost['5~30']+=1
    else:
        time_cost.setdefault('30~',0)
        time_cost['30~']+=1

    count+=1
    if count >= 10000:
        break
print(time_cost)

In [ ]:
time_cost={}
count=0
for item in c.pmdb.order_liuliang.find({'status':4,'channel':'dhst'},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    try:
        for timeline in item['timeline_new']:
            if timeline['status']=='充值中':
                time_1 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
            elif timeline['status']=='充值成功':
                time_2 = time.mktime(time.strptime(timeline['time'],'%Y-%m-%d %H:%M'))
    except :
        continue
            
    cost = time_2-time_1
    if cost <= 60:
        time_cost.setdefault('~1',0)
        time_cost['~1']+=1
    elif 60< cost <= 60*5:
        time_cost.setdefault('1~5',0)
        time_cost['1~5']+=1
    elif 60*5< cost <= 60*30:
        time_cost.setdefault('5~30',0)
        time_cost['5~30']+=1
    else:
        time_cost.setdefault('30~',0)
        time_cost['30~']+=1

    count+=1
    if count >= 10000:
        break
print(time_cost)

In [ ]:
reason_counter={}
count=0
for item in c.pmdb.order_liuliang.find({'channel':'dhst','status':7},{'_id':0,'timeline_new':1}).sort([('_id',-1)]):
    for timeline in item.get('timeline_new',[]):
        if '订单失败' in timeline['status']:
            reason = timeline['status']
            reason_counter.setdefault(reason,0)
            reason_counter[reason]+=1
pprint(reason_counter)

In [ ]:
# 给用户发短信
header = {'Content-Type': 'application/x-www-form-urlencoded'}
sms_url = 'https://api.submail.cn/message/xsend.json'
faild_phone = []
for phone in phone_list[1:]:
    post_data = {
        'appid': '12773',
        'project': 'c8ke44',
        'to': phone,
        'signature':'1324fdded1ccce9f855300215985ab6c'
    }
    resp = requests.post(sms_url,
                         headers=header,
                         data=post_data,
                         verify=False).json()
    if resp['status'] == 'success':
        continue
    faild_phone.append(phone)
    print(phone,resp)

In [ ]:
faild_phone

In [ ]:
tab_data = tablib.Dataset(headers=['订单号','下单到等待退款的间隔时间/分钟'])
start = time.mktime((2018,5,11,0,0,0,0,0,0))*1000
for item in c.pmdb.order_coffee.find({
    'channel':'linqu',
    'appCode':'fdbank',
    'status':{'$in':[7,11]},
    'createdAt':{'$gte':start}
},{'timeline_new':1,'id':1,'_id':0,'createdAt':1}):
    end_time = [timeline['time'] for timeline in item['timeline_new'] if timeline['status'] == '等待退款']
    cost_time = (time.mktime(time.strptime(end_time[0],"%Y-%m-%d %H:%M")) - item['createdAt']/1000)/60
    print(item['id'],round(cost_time,2) if cost_time>0 else round(random.random(),2))
    tab_data.append([item['id'],round(cost_time,2) if cost_time>0 else round(random.random(),2)])
with open('fdbank-下单到等待退款的间隔时间统计.xls','wb') as f:
    f.write(tab_data.xls)

In [64]:
tab_data.csv

'订单号,下单到等待退款的间隔时间/分钟\r\nfdb204595525757394,0.5\r\nfdb204595416213832,10.6\r\nfdb204595413838736,0.91\r\nfdb204595396668268,0.09\r\nfdb204595304147036,0.23\r\nfdb204595081193077,0.21\r\nfdb204594562917596,20.11\r\nfdb204594420048875,51.27\r\nfdb204594301051213,20.4\r\nfdb204594239693547,20.21\r\nfdb204594222916309,21.35\r\nfdb204594087403567,11.57\r\nfdb204594077540398,95.82\r\nfdb204594070741518,0.22\r\nfdb204594048475136,20.53\r\nfdb204594037187029,20.96\r\nfdb204593994080174,122.45\r\nfdb204593989214618,123.05\r\nfdb204593918812017,151.01\r\nfdb204593882390844,158.64\r\nfdb204593857109818,88.85\r\nfdb204593834434342,169.74\r\nfdb204593801257188,171.95\r\nfdb204593758936806,20.35\r\nfdb204593732182177,1.74\r\nfdb204593712635560,183.22\r\nfdb204593519730117,207.76\r\nfdb204593496235399,0.38\r\nfdb204593495580038,204.83\r\nfdb204593476198230,0.18\r\nfdb204593472266063,0.93\r\nfdb204593466367811,0.05\r\nfdb204593434533193,218.6\r\nfdb204593379056855,0.74\r\nfdb204593372142838,229.53\r\nf

In [47]:
round(12.33333,2)

12.33

## 世界杯场次录入和更新

In [ ]:
import pandas as pd
import numpy as np

url = 'https://api.otosaas.com'
# Location of file
Location = r'worldCup_showing.xls'
df = pd.read_excel(Location,
                   dtype={
                       'options': np.str, 
                       'date': np.str,
                       'time':np.str,
                       'score':np.int32,
                       'isContainDogfall':np.bool,
                       'goal':np.str,
                       'winner':np.str,
                       'id':np.str
                   },
                  na_values=[],
                  keep_default_na=False)

for index,item in df.iterrows():
    # 世界杯场次录入
    if not item.id:
        body = {
            'options': item.options.split('-'),
            'date': item.date, 
            'time': item.time,
            'score': item.score, 
            'isContainDogfall': item.isContainDogfall
        }
        resp = requests.put(url+'/worldcup/v1/showing',json=body).json()
        item.id = df.loc[index,'id'] = resp['data']['id']
        print('insert',item.options,resp['data']['id'],resp['code'])

    # 更新场次winner
    if item.id and item['goal']:
        body = {
            "id":item['id'],
            "options":[{"name":name,"goal":goal} for name,goal in zip(item['options'].split('-'),item['goal'].split('-'))],
            "winner":item['winner']
        }
        resp = requests.post(url+'/worldcup/v1/showing',json=body).json()
        print('update',item.date,item.options,item.id,resp['code'])

df.to_excel('worldCup_showing.xls', sheet_name='worldCup_showing',index=False)

'2018-06-16'

In [307]:
yesterday

'2018-06-16'

In [ ]:
# 根据activityId，查询卷发给哪些账户
# 丰收互联 世界杯iPhoneX    活动ID ：1471
# 包商银行  世界杯100元代金券  活动ID：1461
# 包商银行  世界杯200元代金券   活动ID：1459
# 日期-中将手机号-优惠券名称
rdb = c.prdb_activity(0)
mdb = c.pmdb
yesterday = arrow.now().shift(days=-1).strftime("%Y-%m-%d")

activity_list = [{
    "activityIdList":['1461','1459'],
    "appCode":"bsb"
},
# {
#     "activityIdList":['1471'],
#     "appCode":"zjrcu"
# }
]

for activity in activity_list:
    app_code = activity['appCode']
    tab_data = tablib.Dataset(headers='日期-中奖手机号-优惠券名称'.split('-'))
    for activity_id in activity['activityIdList']:
        activity_title = rdb.hget(f'a:{activity_id}','title')
        for key in rdb.scan_iter(f'c:{app_code}:*'):
            for item,timestamp in c.prdb_activity(0).zscan_iter(key):
                if activity_id == item.split('|')[-1]:
                    date = time.strftime("%Y-%m-%d",time.localtime(timestamp))
                    if date !=yesterday:
                        continue
                    user_id = key.split(':')[2]
                    user_customer = mdb.user_customer.find_one({f'customerUser.{app_code}':user_id})
                    phone =user_customer['phone'] if user_customer else user_id
                    tab_data.append([date,phone,activity_title])
                    print([date,phone,activity_title])
    with open('{}-世界杯活动中奖名单-{}.xls'.format(app_code,yesterday),'wb') as f:
        f.write(tab_data.xls)

In [321]:
# 用户中奖概率
tab_data = tablib.Dataset(headers=['用户名','中奖次数','抽奖次数','中奖率'])
user_counter = {}
for item in c.pmdb.order_worldCup.find({'appCode':'zjrcu'}):
    user_counter.setdefault(item['customerUserId'],{'campaign_count':0,'campaign_sum':0})
    user_counter[item['customerUserId']]['campaign_sum']+=1
    if item['campaign_id']:
        user_counter[item['customerUserId']]['campaign_count']+=1
for user_id,counter in user_counter.items():
    tab_data.append([user_id,counter['campaign_count'],counter['campaign_sum'],'{:.2f}'.format(counter['campaign_count']/counter['campaign_sum'])])
with open('用户中奖概率.xls','wb') as f:
    f.write(tab_data.xls)

In [61]:
key = 'o_r_mapper:91ala:ala103077070913625'
rdb = c.srdb_new(0)
# list(.zscan_iter(key))

In [62]:
rdb.smembers(key)

{'ala123077075501147'}

In [81]:
# 修复加油卡订单的channel:ofPay为ofpay!
rdb = c.prdb_paym
mdb = c.pmdb
url='http://sessionapi.localdomain:8010/sm/order/update'

for item in mdb.order_jiayouka.find({'channel':{'$in':['ofpay','ofPay']}}).sort([('createdAt',-1)]):
    order_id = item['id']
    body={'id':order_id,'channel':'ofpay'}
    resp=requests.post(url=url,json=body).json()
    try:
        app_code = resp["data"]["appCode"]
    except:
        continue    
        
    for flow_id in (resp['data'].get('paymentSerialList',[]) or resp['data'].get('paidList',[])):
        flow_key = f'p:{app_code}:{flow_id}'
        if rdb(1).hget(flow_key,'supplier') == 'ofPay':
            rdb(1).hset(flow_key,'supplier','ofpay')
            print(flow_key,resp['message'])

    for flow_id in rdb(0).smembers(f'o_r_mapper:{app_code}:{order_id}'):
        flow_key = f'r:{app_code}:{flow_id}'
        if rdb(2).hget(flow_key,'supplier') == 'ofPay':
            rdb(2).hset(flow_key,'supplier','ofpay')
            print(flow_key,resp['message'])

In [85]:
# 数据统计
with open('/data/logs/kong/2018_07_03_access.log') as f:
    data_log = f.read()

In [102]:
count = 0
for i in range(1,31):
    try:
        with open('2018_06_{:0>2d}_access.log'.format(i)) as f:
            data_log = f.read()
    except:
        break
    for item in data_log.split('\n'):
        if 'GET' in item and 'Java' not in item:
            count+=1
print(count/30)

0


In [ ]:
count = 0
for i in range(100):
    count+=len(c.pmdb.order_lite_list.distinct('userId',{'createdAt':{
        '$gte':(time.time()-86400*100*i)*1000,
        '$lt':(time.time()-86400*100*(i-1))*1000
    }}))
    print(count)
print(count)

In [109]:
# 统计用户购买但未使用的情况信息
mdb = c.pmdb
ardb = c.prdb_activity(0)
filter_config_list = [
    {
        'zjrcu': {
            'orderType': {'$in': ['evoucher']},
            'channel':{'$in': ['otosaas']}
        }
    }
]
counter = {}
for filter_config in filter_config_list:
    for app_code, _filter in filter_config.items():
        counter.setdefault(app_code,{'count':0,'sumPrice':0})
        _filter.update({'status': 4, 'appCode': app_code, 'createdAt': {'$gte': arrow.now().shift(months=-30).timestamp * 1000}})
        for lite_item in mdb.order_lite_list.find(_filter, {'id': 1, '_id': 0, 'orderType': 1}):
            order_data = mdb['order_{}'.format(lite_item['orderType'])].find_one({'id': lite_item['id']})
            if order_data is None:
                continue
            unused_key = 'c:{}:{}'.format(app_code, order_data['userId'])
            expired_key = 'c:{}:{}:expired'.format(app_code, order_data['userId'])
            for created_data, created_time in ardb.zscan_iter(expired_key, match='*|{}'.format(order_data['activity_id'])):
                # created_data: '55637992|c:1580|2185', created_time: 1524886313.0
                counter[app_code]['count']+=1
                counter[app_code]['sumPrice']+=order_data['price']
                break
            threshold_time = arrow.now().timestamp - 86400 * 0
            for created_data, created_time in ardb.zscan_iter(unused_key, match='*|{}'.format(order_data['activity_id'])):
                coupon_id = created_data.split('|')[1]
                coupon_content = ardb.hget('a:{}'.format(coupon_id), 'content')
                coupon_json = json.loads(coupon_content)
                counter[app_code]['count']+=1
                counter[app_code]['sumPrice']+=order_data['price']
                break
#                 if coupon_json['ets'] < threshold_time:
#                     counter[app_code]['count']+=1
#                     counter[app_code]['sumPrice']+=order_data['price']
#                     break

In [128]:
data = {}
for a in range(46):
    for b in range(46-a):
        c = 45-a-b
        result = 5*a*(1+0.1*b)*(1+0.2*c)
        data.setdefault(result,[])
        data[result].append("a={},b={},c={}".format(a,b,c))
data[max(data)]

['a=20,b=10,c=15']